# Ozone forcing


Skeie et al. 2021:

- time series of all depleters, warming included

Individual contributions to present day ozone forcing (estimated from figure 7, Thornhill et al 2021) with no climate contribution:

- N2O +0.04
- VOC +0.11
- NOx +0.14
- CH4 +0.17  (O'Connor et al. says +0.13 or +0.20 depending on method; we use +0.13)
- HC  -0.33
- climate -0.079 W/m2/K  (table 14 other Thornhill paper)
- sum +0.13  (without climate feedack)  (+0.09 with pre-calibrated methane from O'Connor which does not include climate feedback either)
- sum +0.05  (with climate feedback assuming +1.0K 1850-2014; +0.01 if assume O'Connor)

This falls a little short, possibly the use of the ozone radiative kernel means that estimates are not perfect in either Skeie or Thornhill x 2, or that there is some additional non-linearity that kicks into the estimates.

**Possible effect of ACI from ozone: drove a stronger negative forcing than expected - can we fit the residual?**

**look at histSST-piO3 and fit the time series to emissions of NOx, VOC and CO**

In [ ]:
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pooch
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit

In [ ]:
st_df = pd.read_csv('../data/skeie2020/skeie_ozone_strat.csv', index_col="Model")
tr_df = pd.read_csv('../data/skeie2020/skeie_ozone_trop.csv', index_col="Model")

In [ ]:
# Skeie uses 10 year averages: 1920 is 1916-1925 up to 2000. 2007 is 2003-2012. 2010 is 2005-2014.
o3_ukesm = st_df.loc['UKESM1-0-LL'] + tr_df.loc['UKESM1-0-LL']
pl.plot(o3_ukesm)

In [ ]:
temp_ukesm = np.ones((165, 19)) * np.nan
for run in range(19):
    try:
        temp_ukesm[:, run] = pd.read_csv('../data/fredriksen2022/historical/UKESM1-0-LL_historical_r%di1p1f2_anomalies.txt' %(run+1), index_col=0)['tas'].values
    except FileNotFoundError:
        pass

In [ ]:
pl.plot(temp_ukesm);
temp_ukesm_mean = np.nanmean(temp_ukesm, axis=1)
pl.plot(temp_ukesm_mean, color='k')

In [ ]:
climate_response_df = pd.read_csv('../data/fair-calibrations/4xCO2_energy_balance_ebm3.csv')

In [ ]:
natural_forcing_df = pd.read_csv('../data/calibration/natural_forcing.csv', index_col=0)

In [ ]:
f = FAIR(temperature_prescribed=True)

In [ ]:
scenarios = ['ssp245']
configs = ['UKESM1-0-LL']
species, properties = read_properties(filename='../data/species_configs_properties_vanilla.csv')
species.remove('CO2 FFI')      # c-driven run
species.remove('CO2 AFOLU')    # c-driven run
species.remove('Halon-1202')   # not in CMIP6 list of species
species.remove('Contrails')    # not modelled in UKESM, I think
species.remove('NOx aviation') # which renders this redundant
species.remove('Light absorbing particles on snow and ice')  # I believe not modelled in UKESM
del properties['CO2 FFI']
del properties['CO2 AFOLU']
del properties['Halon-1202']
del properties['Contrails']
del properties['NOx aviation']
del properties['Light absorbing particles on snow and ice']
for specie in species:
    if properties[specie]['greenhouse_gas']:
        properties[specie]['input_mode'] = 'concentration'
properties['Land use']['input_mode'] = 'forcing'

In [ ]:
f.define_time(1850, 2015, 1)
f.define_scenarios(scenarios)
f.define_configs(configs)
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])

In [ ]:
initialise(f.forcing, 0)
initialise(f.temperature, 0)

In [ ]:
f.fill_species_configs(filename='../data/species_configs_properties_vanilla.csv')

In [ ]:
f.fill_from_rcmip()

In [ ]:
# override RCMIP natural forcing
fill(f.forcing, np.array(natural_forcing_df.loc[1850:2015, 'solar'])[:, None, None], specie='Solar')
fill(f.forcing, np.array(natural_forcing_df.loc[1850:2015, 'volcanic'])[:, None, None], specie='Volcanic')

In [ ]:
f.emissions[:,0,0,3:10]

In [ ]:
f.emissions.loc[dict(specie='Sulfur', timepoints=1850.5)]

In [ ]:
landuse_erf = pd.read_csv('../data/calibration/landuse_forcing.csv', index_col=0)
f.forcing.loc[dict(specie='Land use')] = landuse_erf.values[:, None]

In [ ]:
# 1850 baselines
for specie in ['Sulfur', 'BC', 'OC', 'NOx', 'VOC', 'CO', 'NH3']:
    f.species_configs['baseline_emissions'].loc[dict(specie=specie)] = f.emissions.loc[dict(specie=specie, timepoints=1850.5, scenario='ssp245')]

for specie in ['CO2', 'CH4', 'N2O', 'CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18', 'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa',
    'HFC-32', 'HFC-365mfc', 'HFC-4310mee']:
    f.species_configs['baseline_concentration'].loc[dict(specie=specie)] = f.concentration.loc[dict(specie=specie, timebounds=1850, scenario='ssp245')]

# check this against the last cell in this script
f.species_configs['baseline_concentration'].loc[dict(specie='Equivalent effective stratospheric chlorine')] = 344.36275911

## rebase volcanic to zero mean
#f.forcing.loc[dict(specie='Volcanic')] = f.forcing.loc[dict(specie='Volcanic')] - f.forcing.loc[dict(specie='Volcanic', scenario='ssp245', config='UKESM1-0-LL')].mean()

In [ ]:
# fill temperature
f.temperature[:-1, ...] = temp_ukesm_mean[:, None, None, None]
f.temperature[-1, ...] = f.temperature[-2, ...]

# pre-calibrated for aerosol (05) and CH4 (07) runs: do not adjust
f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='Sulfur')] = -0.00283793
f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='BC')] = 0.01757433
f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='OC')] = -0.0028512
f.species_configs['aci_scale'].loc[dict(config='UKESM1-0-LL')] = -8.22336281e-01
f.species_configs['aci_shape'].loc[dict(specie='Sulfur')] = 3.01888722e-02
f.species_configs['aci_shape'].loc[dict(specie='BC')] = 5.07111186e-30
f.species_configs['aci_shape'].loc[dict(specie='OC')] = 6.60381587e-16
f.species_configs['forcing_scale'].loc[dict(specie="CH4")] = 1.11547955
f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="CH4")] = 1.27049657e-04
f.species_configs['h2o_stratospheric_factor'].loc[dict(specie="CH4")] = 1.29652705e-01
f.species_configs['erfari_radiative_efficiency'].loc[dict(specie="CH4")] = -5.09377882e-05
f.species_configs['aci_shape'].loc[dict(specie="CH4")] = -2.61186113e-04

# co-calibrated with N2O (10) and CO2 (09) runs:
f.species_configs['forcing_scale'].loc[dict(specie="N2O")] = 1.16044956348451
f.species_configs['forcing_scale'].loc[dict(specie="CO2")] = 1.04321901751203

In [ ]:
ch4 = f.concentration.loc[dict(specie='CH4')].squeeze().values
n2o = f.concentration.loc[dict(specie='N2O')].squeeze().values
eesc = f.concentration.loc[dict(specie='Equivalent effective stratospheric chlorine')].squeeze().values

voc = f.emissions.loc[dict(specie='VOC')].squeeze().values
co = f.emissions.loc[dict(specie='CO')].squeeze().values
nox = f.emissions.loc[dict(specie='NOx')].squeeze().values
print(eesc[-2])

In [ ]:
# values from Thornhill
n2o_eff = 0.04 / (n2o[164]-n2o[0])
voc_eff = 0.11 / (voc[164]-voc[0])
nox_eff = 0.14 / (nox[164]-nox[0])
eesc_eff = -0.33 / (1.13324964e+03)# precalc - also the forcing in the present day has been hand tuned to fit skeie much better
n2o_eff, voc_eff, nox_eff, eesc_eff

In [ ]:
f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="Equivalent effective stratospheric chlorine")] = eesc_eff
f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="N2O")] = n2o_eff
f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="VOC")] = voc_eff
f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="NOx")] = nox_eff
f.species_configs['forcing_temperature_feedback'].loc[dict(specie="Ozone")] = -0.079  # Thornhill et al 2021 feedbacks

In [ ]:
f.run()

In [ ]:
# print('N2O               %+7.5f' % f.forcing.values[-2:-1,0,0,1].mean())
print('Ozone             %+7.5f' % f.forcing.values[-2:-1,0,0,54].mean())
# print('ARI               %+7.5f' % f.forcing.values[-2:-1,0,0,10].mean())
# print('ACI               %+7.5f' % f.forcing.values[-2:-1,0,0,11].mean())
# print('Sum of components %+7.5f' % (f.forcing.values[-2:-1,0,0,1]+f.forcing.values[-2:-1,0,0,9]+f.forcing.values[-2:-1,0,0,10]+f.forcing.values[-2:-1,0,0,11]))
print('Scenario sum      %+7.5f' % f.forcing_sum.values[-2:-1,0,0].mean())

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='Ozone', scenario='ssp245', config='UKESM1-0-LL')])
pl.plot(o3_ukesm.index.map(float), o3_ukesm)

In [ ]:
f.forcing[1, 0, 0, :]

In [ ]:
f.forcing_sum[1, 0, 0]

In [ ]:
f.temperature[:, 0, 0, 0]

In [ ]:
f.concentration.sel(specie='Equivalent effective stratospheric chlorine', timebounds=np.arange(1850, 1853))

In [ ]:
f.concentration.sel(specie='Equivalent effective stratospheric chlorine', timebounds=np.arange(2014, 2016))